In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp experiments

In [ ]:
#| export
import os
import csv
from comet_ml import API, APIExperiment
from pct.hierarchy import PCTHierarchy  # Assuming PCTHierarchy is defined elsewhere


In [ ]:
#| export
class CometExperimentManager:
    def __init__(self, api_key: str, workspace: str):
        self.api = API(api_key)
        self.workspace = workspace

    def get_all_artifacts_sorted(self):
        """Retrieve all artifacts and sort them by source experiment key."""
        artifacts = self.api.get_artifacts(workspace=self.workspace)
        return sorted(artifacts, key=lambda artifact: artifact['source_experiment_key'])

    def get_experiments_by_metrics(self, project_name: str, score_threshold: float, reward_threshold: float):
        """
        Retrieve experiments for a project where the metric 'score' is less than
        score_threshold and 'reward' is greater than or equal to reward_threshold.
        """
        experiments = self.api.get(project_name=project_name, workspace=self.workspace)
        filtered_experiments = [
            exp for exp in experiments
            if exp.get_metrics_summary('score') < score_threshold and
               exp.get_metrics_summary('reward') >= reward_threshold
        ]
        return filtered_experiments

    def get_artifact_name(self, experiment: APIExperiment):
        """Retrieve the name of an artifact from an experiment."""
        artifacts = experiment.get_artifacts()
        return artifacts[0]['artifact_name'] if artifacts else None

    def download_and_run_artifact(self, artifact_name: str, num_runs: int, seeds: list[int]):
        """
        Download an artifact to '/tmp/artifacts/' and run PCTHierarchy.run_from_file
        with the artifact filename, returning the score value for each run.
        """
        artifact_path = f"/tmp/artifacts/{artifact_name}"
        os.makedirs(os.path.dirname(artifact_path), exist_ok=True)
        self.api.download_artifact(artifact_name, artifact_path)

        scores = []
        for seed in seeds:
            score = PCTHierarchy.run_from_file(artifact_path, seed=seed)
            scores.append(score)
        return scores

    def run_experiments_and_record_results(self, experiments: list[APIExperiment], num_runs: int, output_csv: str):
        """
        Run each experiment a given number of times and record the score and the
        number of times the reward is 100, -100, or something else. Save results to a CSV file.
        """
        results = []
        for experiment in experiments:
            artifact_name = self.get_artifact_name(experiment)
            if not artifact_name:
                continue

            scores = self.download_and_run_artifact(artifact_name, num_runs, seeds=range(num_runs))
            reward_counts = {'100': 0, '-100': 0, 'other': 0}

            for score in scores:
                if score == 100:
                    reward_counts['100'] += 1
                elif score == -100:
                    reward_counts['-100'] += 1
                else:
                    reward_counts['other'] += 1

            results.append({
                'experiment_key': experiment.id,
                'artifact_name': artifact_name,
                'scores': scores,
                'reward_100': reward_counts['100'],
                'reward_-100': reward_counts['-100'],
                'reward_other': reward_counts['other']
            })

        # Save results to CSV
        with open(output_csv, mode='w', newline='') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=['experiment_key', 'artifact_name', 'scores', 'reward_100', 'reward_-100', 'reward_other'])
            writer.writeheader()
            writer.writerows(results)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()